### Socket Programming-Threads

In [ ]:
import threading
import socket
import pickle
import hashlib
from ecdsa import SigningKey

In [ ]:
class Transaction:
    
    def __init__(self, fromm, to, amount, signature):
        self.fromm = fromm
        self.to = to
        self.amount = amount
        self.signature = signature
    
    def get_string(self):
        trans_string_obj = str(self.fromm)+str(self.to)+str(self.amount)
        return trans_string_obj

In [ ]:
class Block:
    
    
    def __init__(self, trans_obj):
        self.block_id = 0
        self.nounce = 0
        self.transaction = trans_obj
        self.previous_hash = None
    
    def get_current_hash(self):
        hash_val = hashlib.sha256((self.get_transaction_string_obj()).encode('utf-8'))
        return hash_val
    
    def get_transaction_string_obj(self):
        trans_string = str()
        for transaction in self.transaction:
            trans_string = trans_string + transaction.get_string()
        trans_string = trans_string + str(self.balance) + str(self.nounce) \
            + str(self.previous_hash)
        return trans_string

In [ ]:
# client side

class Peer:
    
    
    def __init__(self, ip):
        self.MANAGER_IP = '192.168.1.106'
        self.MANAGER_PORT = 8827
        self.server_port = 5527
        self.client_ip = ip
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)
        self.blockchain = list()
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        self.peers = list()
        self.balance = 0
       
    
    def start_server(self):
        self.server_socket.bind((self.client_ip, self.server_port))
        self.server_socket.listen(100)
        while True:
            client_peer, address = self.server_socket.accept()
            # actions
            data = client_peer.recv(4096).decode('utf-8')
            # add  
            if data.startswith('add-peer'):
                self.peers.add(data.split()[1])
            elif data.startswith('remove-peer'):
                self.peers.remove(data.split()[1])
            else:
                transaction_obj = pickle.loads(client_peer.recv(4096))
                new_block = Block(transaction_obj)
                self.blockchain.append(new_block)
        self.server_socket.close()
      
    
    def _stop_server(self):
        self.stop_server = True
      
    
    def _inform_manager_add(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
        peers = pickle.loads(sock.recv(4096))
        self.peers = peers
    
    
    def _inform_manager_delete(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
    
    
    def leave_network(self):
        return self._inform_manager_delete(f"remove {self.client_ip}")
        
        
    def join_network(self):
        return self._inform_manager_add(f'add {self.client_ip}')
    
    
    def update_balance(self, amount):
        self.balance = amount
    
    def do_transaction(self, fromm, to, amount):
        if amount > self.balance:
            print("Not allowed")
            #actions
        else :
            self.balance -= amount
            message = str(fromm) + str(to) + str(amount)
            public_key = SigningKey.generate() # uses NIST192p
            secret_key = public_key.get_verifying_key()
            signature = public_key.sign(message.encode('utf-8'))
            transaction_obj = Transaction(fromm, to, amount, signature)
            new_block = Block(transaction_obj)
            self.blockchain.append(new_block)
            for peer in self.peers:
                sock = socket.socket()
                try:
                    sock.connect((peer, self.server_port))
                    sock.send("something".encode("utf-8"))
                    sock.send(pickle.dumps(transaction_obj))
                except Exception as e:
                    print(peer)
                    print(e)
                finally:
                    sock.close()
    def mine_transaction(self):
        pass

In [ ]:
client = Peer('192.168.1.110')

In [ ]:
client.join_network()

In [ ]:
client.leave_network()

In [ ]:
client._stop_server()

In [ ]:
client.peers

In [ ]:
client.update_balance(1000)

In [ ]:
client.do_transaction('192.168.1.110', '192.168.1.118', 100)

In [ ]:
client.blockchain

In [1]:
import time
import hashlib
import json
import uuid

NANOSECONDS = 1
MICROSECONDS = 1000 * NANOSECONDS
MILLISECONDS = 1000 * MICROSECONDS
SECONDS = 1000 * MILLISECONDS

MINE_RATE = 4 * SECONDS

STARTING_BALANCE = 1000

MINING_REWARD = 50
MINING_REWARD_INPUT = { 'address': '*--official-mining-reward--*' }

In [2]:
'''CRYPTO'''

def crypto_hash(*args):
    """
    Return a sha-256 hash of the given arguments.
    """
    stringified_args = sorted(map(lambda data: json.dumps(data), args))
    joined_data = ''.join(stringified_args)

    return hashlib.sha256(joined_data.encode('utf-8')).hexdigest()

def main():
    print(f"crypto_hash('one', 2, [3]): {crypto_hash('one', 2, [3])}")
    print(f"crypto_hash(2, 'one', [3]): {crypto_hash(2, 'one', [3])}")

if __name__ == '__main__':
    main()


crypto_hash('one', 2, [3]): 49d135ee795768472bd5f9e5d11c3982e6bdeae55bc86a0f4351654e3d4e8b2a
crypto_hash(2, 'one', [3]): 49d135ee795768472bd5f9e5d11c3982e6bdeae55bc86a0f4351654e3d4e8b2a


In [3]:
HEX_TO_BINARY_CONVERSION_TABLE = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'a': '1010',
    'b': '1011',
    'c': '1100',
    'd': '1101',
    'e': '1110',
    'f': '1111'
}

def hex_to_binary(hex_string):
    binary_string = ''

    for character in hex_string:
        binary_string += HEX_TO_BINARY_CONVERSION_TABLE[character]

    return binary_string

def main():
    number = 451
    hex_number = hex(number)[2:]
    print(f'hex_number: {hex_number}')

    binary_number = hex_to_binary(hex_number)
    print(f'binary_number: {binary_number}')

    original_number = int(binary_number, 2)
    print(f'original_number: {original_number}')

    hex_to_binary_crypto_hash = hex_to_binary(crypto_hash('test-data'))
    print(f'hex_to_binary_crypto_hash: {hex_to_binary_crypto_hash}')

if __name__ == '__main__':
    main()

hex_number: 1c3
binary_number: 000111000011
original_number: 451
hex_to_binary_crypto_hash: 1000011110000111100110000100010011000000001010110100001011001011111110101111110000101001100110010001101001110111011001010110010111111001101101101011011100001001101101100101101011110011111100101001000111011011000010101001000000000000010011001100111110101011


In [4]:
'''BLOCK.PY'''

GENESIS_DATA = {
    'timestamp': 1,
    'last_hash': 'genesis_last_hash',
    'hash': 'genesis_hash',
    'data': [],
    'difficulty': 3,
    'nonce': 'genesis_nonce'
}

class Block:
    """
    Block: a unit of storage.
    Store transactions in a blockchain that supports a cryptocurrency.
    """
    def __init__(self, timestamp, last_hash, hash, data, difficulty, nonce):
        self.timestamp = timestamp
        self.last_hash = last_hash
        self.hash = hash
        self.data = data
        self.difficulty = difficulty
        self.nonce = nonce

    def __repr__(self):
        return (
            'Block('
            f'timestamp: {self.timestamp}, '
            f'last_hash: {self.last_hash}, '
            f'hash: {self.hash}, '
            f'data: {self.data}, '
            f'difficulty: {self.difficulty}, '
            f'nonce: {self.nonce})'
        )

    def __eq__(self, other):
        return self.__dict__ == other.__dict__

    def to_json(self):
        """
        Serialize the block into a dictionary of its attributes
        """
        return self.__dict__

    @staticmethod
    def mine_block(last_block, data):
        """
        Mine a block based on the given last_block and data, until a block hash
        is found that meets the leading 0's proof of work requirement.
        """
        timestamp = time.time()
        last_hash = last_block.hash
        difficulty = Block.adjust_difficulty(last_block, timestamp)
        nonce = 0
        hash = crypto_hash(timestamp, last_hash, data, difficulty, nonce)

        while hex_to_binary(hash)[0:difficulty] != '0' * difficulty:
            nonce += 1
            timestamp = time.time()
            difficulty = Block.adjust_difficulty(last_block, timestamp)
            hash = crypto_hash(timestamp, last_hash, data, difficulty, nonce)

        return Block(timestamp, last_hash, hash, data, difficulty, nonce)

    @staticmethod
    def genesis():
        """
        Generate the genesis block.
        """
        return Block(**GENESIS_DATA)

    @staticmethod
    def from_json(block_json):
        """
        Deserialize a block's json representation back into a block instance.
        """
        return Block(**block_json)

    @staticmethod
    def adjust_difficulty(last_block, new_timestamp):
        """
        Calculate the adjusted difficulty according to the MINE_RATE.
        Increase the difficulty for quickly mined blocks.
        Decrease the difficulty for slowly mined blocks.
        """
        if (new_timestamp - last_block.timestamp) < MINE_RATE:
            return last_block.difficulty + 1

        if (last_block.difficulty - 1) > 0:
            return last_block.difficulty - 1

        return 1

    @staticmethod
    def is_valid_block(last_block, block):
        """
        Validate block by enforcing the following rules:
          - the block must have the proper last_hash reference
          - the block must meet the proof of work requirement
          - the difficulty must only adjust by 1
          - the block hash must be a valid combination of the block fields
        """
        if block.last_hash != last_block.hash:
            raise Exception('The block last_hash must be correct')

        if hex_to_binary(block.hash)[0:block.difficulty] != '0' * block.difficulty:
            raise Exception('The proof of work requirement was not met')

        if abs(last_block.difficulty - block.difficulty) > 1:
            raise Exception('The block difficulty must only adjust by 1')

        reconstructed_hash = crypto_hash(
            block.timestamp,
            block.last_hash,
            block.data,
            block.nonce,
            block.difficulty
        )

        if block.hash != reconstructed_hash:
            raise Exception('The block hash must be correct')

def main():
    genesis_block = Block.genesis()
    bad_block = Block.mine_block(genesis_block, 'foo')
    bad_block.last_hash = 'evil_data'

    try:
        Block.is_valid_block(genesis_block, bad_block)
    except Exception as e:
        print(f'is_valid_block: {e}')

if __name__ == '__main__':
    main()


is_valid_block: The block last_hash must be correct


In [5]:
'''BLOCKCHAIN.PY'''

class Blockchain:
    """
    Blockchain: a public ledger of transactions.
    Implemented as a list of blocks - data sets of transactions
    """
    def __init__(self):
        self.chain = [Block.genesis()]

    def add_block(self, data):
        self.chain.append(Block.mine_block(self.chain[-1], data))

    def __repr__(self):
        return f'Blockchain: {self.chain}'

    def replace_chain(self, chain):
        """
        Replace the local chain with the incoming one if the following applies:
          - The incoming chain is longer than the local one.
          - The incoming chain is formatted properly.
        """
        if len(chain) <= len(self.chain):
            raise Exception('Cannot replace. The incoming chain must be longer.')

        try:
            Blockchain.is_valid_chain(chain)
        except Exception as e:
            raise Exception(f'Cannot replace. The incoming chain is invalid: {e}')

        self.chain = chain

    def to_json(self):
        """
        Serialize the blockchain into a list of blocks.
        """
        return list(map(lambda block: block.to_json(), self.chain))

    @staticmethod
    def from_json(chain_json):
        """
        Deserialize a list of serialized blocks into a Blokchain instance.
        The result will contain a chain list of Block instances.
        """
        blockchain = Blockchain()
        blockchain.chain = list(
            map(lambda block_json: Block.from_json(block_json), chain_json)
        )

        return blockchain

    @staticmethod
    def is_valid_chain(chain):
        """
        Validate the incoming chain.
        Enforce the following rules of the blockchain:
          - the chain must start with the genesis block
          - blocks must be formatted correctly
        """
        if chain[0] != Block.genesis():
            raise Exception('The genesis block must be valid')

        for i in range(1, len(chain)):
            block = chain[i]
            last_block = chain[i-1]
            Block.is_valid_block(last_block, block)

        Blockchain.is_valid_transaction_chain(chain)

    @staticmethod
    def is_valid_transaction_chain(chain):
        """
        Enforce the rules of a chain composed of blocks of transactions.
            - Each transaction must only appear once in the chain.
            - There can only be one mining reward per block.
            - Each transaction must be valid.
        """
        transaction_ids = set()

        for i in range(len(chain)):
            block = chain[i]
            has_mining_reward = False

            for transaction_json in block.data:
                transaction = Transaction.from_json(transaction_json)

                if transaction.id in transaction_ids:
                    raise Exception(f'Transaction {transaction.id} is not unique')

                transaction_ids.add(transaction.id)

                if transaction.input == MINING_REWARD_INPUT:
                    if has_mining_reward:
                        raise Exception(
                            'There can only be one mining reward per block. '\
                            f'Check block with hash: {block.hash}'
                        )

                    has_mining_reward = True
                else:
                    historic_blockchain = Blockchain()
                    historic_blockchain.chain = chain[0:i]
                    historic_balance = Wallet.calculate_balance(
                        historic_blockchain,
                        transaction.input['address']
                    )

                    if historic_balance != transaction.input['amount']:
                        raise Exception(
                            f'Transaction {transaction.id} has an invalid '\
                            'input amount'
                        )

                Transaction.is_valid_transaction(transaction)

def main():
    blockchain = Blockchain()
    blockchain.add_block('one')
    blockchain.add_block('two')

    print(blockchain)
    print(f'blockchain.py ___name__: {__name__}')

if __name__ == '__main__':
    main()


Blockchain: [Block(timestamp: 1, last_hash: genesis_last_hash, hash: genesis_hash, data: [], difficulty: 3, nonce: genesis_nonce), Block(timestamp: 1574228120.035286, last_hash: genesis_hash, hash: 02755a6c7a11c79346f7c50b065888f5dbd2fe52f70b3c334b9aaeb914773cb9, data: one, difficulty: 4, nonce: 7), Block(timestamp: 1574228120.035959, last_hash: 02755a6c7a11c79346f7c50b065888f5dbd2fe52f70b3c334b9aaeb914773cb9, hash: 0074bba70ab1df197a7902d1374f9bd1d5c7bd5c12e8c8003ccf71f76a2afb4a, data: two, difficulty: 5, nonce: 23)]
blockchain.py ___name__: __main__


In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric.utils import (
    encode_dss_signature,
    decode_dss_signature
)
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.exceptions import InvalidSignature

class Wallet:
    """
    An individual wallet for a miner.
    Keeps track of the miner's balance.
    Allows a miner to authorize transactions.
    """
    def __init__(self, blockchain=None):
        self.blockchain = blockchain
        self.address = str(uuid.uuid4())[0:8]
        self.private_key = ec.generate_private_key(
            ec.SECP256K1(),
            default_backend()
        )
        self.public_key = self.private_key.public_key()
        self.serialize_public_key()

    @property
    def balance(self):
        return Wallet.calculate_balance(self.blockchain, self.address)

    def sign(self, data):
        """
        Generate a signature based on the data using the local private key.
        """
        return decode_dss_signature(self.private_key.sign(
            json.dumps(data).encode('utf-8'),
            ec.ECDSA(hashes.SHA256())
        ))

    def serialize_public_key(self):
        """
        Reset the public key to its serialized version.
        """
        self.public_key = self.public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ).decode('utf-8')

    @staticmethod
    def verify(public_key, data, signature):
        """
        Verify a signature based on the original public key and data.
        """
        deserialized_public_key = serialization.load_pem_public_key(
            public_key.encode('utf-8'),
            default_backend()
        )

        (r, s) = signature

        try:
            deserialized_public_key.verify(
                encode_dss_signature(r, s),
                json.dumps(data).encode('utf-8'),
                ec.ECDSA(hashes.SHA256())    
            )
            return True
        except InvalidSignature:
            return False

    @staticmethod
    def calculate_balance(blockchain, address):
        """
        Calculate the balance of the given address considering the transaction
        data within the blockchain.

        The balance is found by adding the output values that belong to the
        address since the most recent transaction by that address.
        """
        balance = STARTING_BALANCE

        if not blockchain:
            return balance

        for block in blockchain.chain:
            for transaction in block.data:
                if transaction['input']['address'] == address:
                    # Any time the address conducts a new transaction it resets
                    # its balance
                    balance = transaction['output'][address]
                elif address in transaction['output']:
                    balance += transaction['output'][address]

        return balance

def main():
    wallet = Wallet()
    print(f'wallet.__dict__: {wallet.__dict__}')

    data = { 'foo': 'bar' }
    signature = wallet.sign(data)
    print(f'signature: {signature}')

    should_be_valid = Wallet.verify(wallet.public_key, data, signature)
    print(f'should_be_valid: {should_be_valid}')

    should_be_invalid = Wallet.verify(Wallet().public_key, data, signature)
    print(f'should_be_invalid: {should_be_invalid}')

if __name__ == '__main__':
    main()


wallet.__dict__: {'blockchain': None, 'address': '3b17cce5', 'private_key': <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey object at 0x1051d2898>, 'public_key': '-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEybGXciBO0iaCU5GQmkuFZRKPr+JwxP+Q\noFMiia0XLHGX4zrCbj4syg0xdT7IwgKpLtvdPBfpzCd8dtvv7+HhMg==\n-----END PUBLIC KEY-----\n'}
signature: (20321488166393463542564100748347838811699021173842873483568090947407964812506, 22966861042422747261562484899481810895872057361485735081383381962498013457466)
should_be_valid: True
should_be_invalid: False


In [7]:
'''TRANSACTION POOL'''
class TransactionPool:
    def __init__(self):
        self.transaction_map = {}

    def set_transaction(self, transaction):
        """
        Set a transaction in the transaction pool.
        """
        self.transaction_map[transaction.id] = transaction


    def existing_transaction(self, address):
        """
        Find a transaction generated by the address in the transaction pool
        """
        for transaction in self.transaction_map.values():
            if transaction.input['address'] == address:
                return transaction
    
    @staticmethod
    def transaction_data(self):
        """
        Return the transactions of thje transaction pool represented in their
        json serialized form.
        """
        return list(map(
            lambda transaction: transaction.to_json(),
            self.transaction_map.values()
        ))

    def clear_blockchain_transactions(self, blockchain):
        """
        Delete blockchain recorded transactions from the transaction pool.
        """
        for block in blockchain.chain:
            for transaction in block.data:
                try:
                    del self.transaction_map[transaction['id']]
                except KeyError:
                    pass

In [8]:
'''Transaction'''

class Transaction:
    """
    Document of an exchange in currency from a sender to one
    or more recipients.
    """
    def __init__(
        self,
        sender_wallet=None,
        recipient=None,
        amount=None,
        id=None,
        output=None,
        input=None
    ):
        self.id = id or str(uuid.uuid4())[0:8]
        self.output = output or self.create_output(
            sender_wallet,
            recipient,
            amount
        )
        self.input = input or self.create_input(sender_wallet, self.output)

    def create_output(self, sender_wallet, recipient, amount):
        """
        Structure the output data for the transaction.
        """
        if amount > sender_wallet.balance:
            raise Exception('Amount exceeds balance')

        output = {}
        output[recipient] = amount
        output[sender_wallet.address] = sender_wallet.balance - amount

        return output

    def create_input(self, sender_wallet, output):
        """
        Structure the input data for the transaction.
        Sign the transaction and include the sender's public key and address
        """
        return {
            'timestamp': time.time(),
            'amount': sender_wallet.balance,
            'address': sender_wallet.address,
            'public_key': sender_wallet.public_key,
            'signature': sender_wallet.sign(output)
        }

    def update(self, sender_wallet, recipient, amount):
        """
        Update the transaction with an existing or new recipient.
        """
        if amount > self.output[sender_wallet.address]:
            raise Exception('Amount exceeds balance')

        if recipient in self.output:
            self.output[recipient] = self.output[recipient] + amount
        else:
            self.output[recipient] = amount

        self.output[sender_wallet.address] = \
            self.output[sender_wallet.address] - amount

        self.input = self.create_input(sender_wallet, self.output)

    def to_json(self):
        """
        Serialize the transaction.
        """
        return self.__dict__

    @staticmethod
    def from_json(transaction_json):
        """
        Deserialize a transaction's json representation back into a
        Transaction instance
        """
        return Transaction(**transaction_json)

    @staticmethod
    def is_valid_transaction(transaction):
        """
        Validate a transaction.
        Raise an exception for invalid transactions.
        """
        if transaction.input == MINING_REWARD_INPUT:
            if list(transaction.output.values()) != [MINING_REWARD]:
                raise Exception('Invalid mining reward')
            return

        output_total = sum(transaction.output.values())

        if transaction.input['amount'] != output_total:
            raise Exception('Invalid transaction output values')

        if not Wallet.verify(
            transaction.input['public_key'],
            transaction.output,
            transaction.input['signature']
        ):
            raise Exception('Invalid signature')

    @staticmethod
    def reward_transaction(miner_wallet):
        """
        Generate a reward transaction that award the miner.
        """
        output = {}
        output[miner_wallet.address] = MINING_REWARD

        return Transaction(input=MINING_REWARD_INPUT, output=output)

def main():
    transaction = Transaction(Wallet(), 'recipient', 15)
    print(f'transaction.__dict__: {transaction.__dict__}')

    transaction_json = transaction.to_json()
    restored_transaction = Transaction.from_json(transaction_json)
    print(f'restored_transaction.__dict__: {restored_transaction.__dict__}')

if __name__ == '__main__':
    main()

transaction.__dict__: {'id': 'd272e428', 'output': {'recipient': 15, '61a8bbb5': 985}, 'input': {'timestamp': 1574228148.869538, 'amount': 1000, 'address': '61a8bbb5', 'public_key': '-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEvmV7qm3/Sn0eh62ZQ62zgjPWEC6ac61u\nMM5TfzDpuelOB2KKrAWEZgO7o0G90chsRoIufEUl5d7SufK9D/920A==\n-----END PUBLIC KEY-----\n', 'signature': (32722915349024641626951775696052656306619648601863628742960209932517026445348, 73704997044913230188297302502669516990687971483103647679368073104473098702749)}}
restored_transaction.__dict__: {'id': 'd272e428', 'output': {'recipient': 15, '61a8bbb5': 985}, 'input': {'timestamp': 1574228148.869538, 'amount': 1000, 'address': '61a8bbb5', 'public_key': '-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEvmV7qm3/Sn0eh62ZQ62zgjPWEC6ac61u\nMM5TfzDpuelOB2KKrAWEZgO7o0G90chsRoIufEUl5d7SufK9D/920A==\n-----END PUBLIC KEY-----\n', 'signature': (327229153490246416269517756960526563066196486018636287429602099325170264453

In [17]:
# client side

class Peer:
    
    def __init__(self, ip):
        self.MANAGER_IP = '192.168.1.106'
        self.MANAGER_PORT = 8827
        self.server_port = 5527
        self.client_ip = ip
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM, 0)
        self.server_thread = threading.Thread(target=self.start_server)
        self.server_thread.daemon = True
        self.server_thread.start()
        self.stop_server = False
        self.peers = list()
        self.blockchain = Blockchain()
        self.wallet = Wallet(self.blockchain)
        self.transaction_pool = TransactionPool()
       
    
    def start_server(self):
        self.server_socket.bind((self.client_ip, self.server_port))
        self.server_socket.listen(100)
        while True:
            client_peer, address = self.server_socket.accept()
            # actions
            data = client_peer.recv(4096).decode('utf-8')
            # add  
            if data.startswith('add-peer'):
                self.peers.add(data.split()[1])
            
            elif data.startswith('remove-peer'):
                self.peers.remove(data.split()[1])
            
            elif data.startswith('sync-chain'):
                blockchain_recieved = pickle.loads(client_peer.recv(4096))
                self.blockchain.replace_chain(blockchain_recieved)
            
            elif data.startswith('add-block'):
                block = client_peer.recv(4096)
                potential_chain = self.blockchain.chain[:]
                potential_chain.append(block)
                try:
                    self.blockchain.replace_chain(potential_chain)
                    self.transaction_pool.clear_blockchain_transactions(
                        self.blockchain
                    )
                    sock = socket.socket()
                    sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
                    sock.send("update-transactionpool".encode('utf-8'))
                    sock.send(pickle.dumps(transaction_pool))
                    print('\n -- Successfully replaced the local chain')
                except Exception as e:
                    print(f'\n -- Did not replace chain: {e}')
                finally:
                    sock.close()
            
            elif data.startswith('add-transaction'):
                new_transaction = client_peer.recv(4096)
                self.transaction_pool.set_transaction(pickle.loads(new_transaction))
            
            elif data.startswith('update-transactionpool'):
                new_transactionpool = client_peer.recv(4096)
                self.transaction_pool = pickle.loads(new_transactionpool)
                
            self.server_socket.close()
      
    
    def _stop_server(self):
        self.stop_server = True
      
    
    def _inform_manager_add(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
        peers = pickle.loads(sock.recv(4096))
        self.transaction_pool = pickle.loads(sock.recv(4096))
        self.peers = peers
    
    
    def _inform_manager_delete(self, message):
        sock = socket.socket()
        sock.connect((self.MANAGER_IP, self.MANAGER_PORT))
        sock.send(message.encode('utf-8'))
    
    
    def leave_network(self):
        return self._inform_manager_delete(f"remove {self.client_ip}")
        
        
    def join_network(self):
        return self._inform_manager_add(f'add {self.client_ip}')
    
    
    def show_blockchain(self):
        print(self.blockchain.to_json())
    
    
    def _inform_peers(self, message, data=None):
        for peer in self.peers:
            sock.connect((peer, self.server_port))
            sock.send(message.encode("utf-8"))
            if data:
                sock.send(pickle.dumps(data))
        sock.close()
    
    def sync_chain(self):
        for peer in self.peers:
            sock = socket.socket()
            try:
                self._inform_peers("sync-chain", self.blockchain)
            except Exception as e:
                print(peer)
                print(e)
            finally:
                sock.close()
    
    def mine_transaction(self):
        transaction_data = transaction_pool.transaction_data()
        print(transaction_data)
        transaction_data.append(Transaction.reward_transaction(wallet).to_json())
        blockchain.add_block(transaction_data)
        block = blockchain.chain[-1]
        try:
            self._inform_peers("add-block", block)
            transaction_pool.clear_blockchain_transactions(blockchain)
            self._inform_peers("update-transactionpool", transaction_pool)
        except Exception as e:
            print(peer)
            print(e)
        finally:
            sock.close()
    
    
    def wallet_transact(self, transaction_data):
        transaction = transaction_pool.existing_transaction(wallet.address)
        if transaction:
            transaction.update(
                wallet,
                transaction_data['recipient'],
                transaction_data['amount']
            )
        else:
            transaction = Transaction(
                wallet,
                transaction_data['recipient'],
                transaction_data['amount']
            )
        try:
            transaction_pool.set_transaction(transaction)
            self._inform_peers("add-transaction", transaction)
        except Exception as e:
            print(e)
        finally:
            sock.close()